In [1]:
# Library Import
import os 
import pandas as pd
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

pd.options.mode.use_inf_as_na = True

In [2]:
# Parameters

Host = "https://strategenics.atlassian.net/"
Username = "mike.burns@strategenics.com.au"
Password = "ATATT3xFfGF0wcJfZvitiiWKYW_9UCdV2r1dWJE6O8a7ZxFRQUU7FVnBkdYlxNx9PcEn_tbHM-tdxz4DZedCozWuNUSI46DcZDPMKpXq0QDgiWomXTEkzDjVZfVa4oRjADfcbgebgME5skwsc5Ue4doiLA1OpiIyxzP1gzXAzNcfYSniUnKKoqI=2FD01AC0"
ValidProjectCategories = ["'Customer Delivery Projects'"]

In [3]:
# Functions
import base64
import pandas as pd
import re 
import requests
from functools import reduce

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        

        response = requests.post(Host + url, headers = headers, json = defaultContents)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes):
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = fnExpandColumn(df, "projects", None)
    df = df[["issuetypes"]]
    df = df.explode("issuetypes")
    df = fnExpandColumn(df, "issuetypes", None)
    df = df[["fields"]]
    df = fnExpandColumn(df, "fields", None)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    df = pd.DataFrame(values)
    df = df.sort_values("fieldId")
    return df 

def fnGetValidProjectKeys():
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    df = fnAPI(ApiCall)
    df = fnExpandColumn(df, "projectCategory")
    df = df.loc[df['projectCategory name'].isin(ValidProjectCategories) | ("'" + df['projectCategory name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnSentenceCase(s):
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]).replace("0", "")

def fnAPI(webRequestDelegate, startAt = 0): 
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df
    
def fnExpandColumn(df:pd.DataFrame, colName:str, prefix:str = "Prefix "):
    if prefix == "Prefix ":
        prefix = colName + " "
    if prefix == None: 
        df = pd.concat([df.drop(colName, axis=1), df[colName].apply(lambda x: pd.Series(x))], axis=1)
    else:
        df = pd.concat([df.drop(colName, axis=1), df[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))], axis=1)
    return df

In [46]:
# Issues Tasks

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC" 

fields = fnGetIssueTypeFields(ExpectedIssueTypes)

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
else:
	df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
	df = df.explode("issues")
	df = fnExpandColumn(df, "issues", None)
	df = df.drop(["expand", "self"], axis=1)
	df = fnExpandColumn(df, "fields", None)
	df = df.drop(["components", "attachment"], axis=1)
	df = df.rename( columns=dict( zip ( fields.fieldId, fields.name )) )
	df = df.convert_dtypes().infer_objects()
	df["id"] = df["id"].astype('Int64')

	display(df.dtypes)
	display(df)

id                          Int64
key                string[python]
Issue Type                 object
Project                    object
Description        string[python]
Fix versions               object
Resolution                 object
Tempo Customer             object
Sprint                     object
Epic Link          string[python]
Resolution Date    string[python]
Summary            string[python]
Last Viewed        string[python]
Created            string[python]
Reporter                   object
Priority                   object
Story Points               object
Labels                     object
Account                    object
Start date         string[python]
Due date           string[python]
Linked Issues              object
Assignee                   object
Updated            string[python]
Status                     object
Parent                     object
dtype: object

,id,key,Issue Type,Project,Description,Fix versions,Resolution,Tempo Customer,Sprint,Epic Link,...,Story Points,Labels,Account,Start date,Due date,Linked Issues,Assignee,Updated,Status,Parent
0,50738,SAT-67,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,<NA>,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,<NA>,...,None,[],"{'id': 35, 'value': 'CSIRO - Billable'}",2023-07-21,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,2023-07-24T15:57:44.935+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
0,50729,SAT-58,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,<NA>,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,<NA>,...,None,[],"{'id': 34, 'value': 'Strategenics - Product De...",2023-07-21,<NA>,"[{'id': '36055', 'self': 'https://strategenics...",{'self': 'https://strategenics.atlassian.net/r...,2023-07-24T15:52:08.049+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
1,50733,SAT-62,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,<NA>,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,<NA>,...,None,[],"{'id': 34, 'value': 'Strategenics - Product De...",2023-07-21,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,2023-07-24T15:39:41.773+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
1,50785,SAT-106,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,<NA>,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,<NA>,...,None,[],"{'id': 34, 'value': 'Strategenics - Product De...",2023-07-21,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,2023-07-24T15:32:33.360+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
2,50794,SAT-111,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,None,<NA>,...,None,[],"{'id': 33, 'value': 'Brisbane City Council - N...",2023-07-21,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,2023-07-24T15:27:53.013+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,44369,TRC-320,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,Review the current business requirements and e...,[{'self': 'https://strategenics.atlassian.net/...,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,None,TRC-1,...,None,[],"{'id': 8, 'value': 'PMIS'}",2021-02-17,2021-04-01,[],None,2023-07-20T17:39:49.389+1000,{'self': 'https://strategenics.atlassian.net/r...,"{'id': '44004', 'key': 'TRC-1', 'self': 'https..."
54,44525,TRC-331,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,None,TRC-330,...,None,[],"{'id': 8, 'value': 'PMIS'}",2021-04-07,<NA>,[],None,2023-07-20T17:39:49.212+1000,{'self': 'https://strategenics.atlassian.net/r...,"{'id': '44524', 'key': 'TRC-330', 'self': 'htt..."
55,44527,TRC-333,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,None,TRC-330,...,None,[],"{'id': 8, 'value': 'PMIS'}",2021-04-07,<NA>,[],None,2023-07-20T17:39:48.984+1000,{'self': 'https://strategenics.atlassian.net/r...,"{'id': '44524', 'key': 'TRC-330', 'self': 'htt..."
55,44526,TRC-332,{'self': 'https://strategenics.atlassian.net/r...,{'self': 'https://strategenics.atlassian.net/r...,Request access to TRC systems for Radhika,[],{'self': 'https://strategenics.atlassian.net/r...,{'se

In [26]:
# Issues Task Components

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") and component is not empty ORDER BY updatedDate DESC" 

fields = fnGetIssueTypeFields(ExpectedIssueTypes)
fields = fields[ fields['name'] == 'Components']

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
else:
	df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
	df = df.explode("issues")
	df = fnExpandColumn(df, "issues", None)
	df = df.drop(["expand", "self"], axis=1)
	df = fnExpandColumn(df, "fields", None)
	df = df.explode("components")
	df = fnExpandColumn(df, "components")
	df = df.drop(["components self"], axis=1)
	df = df.convert_dtypes().infer_objects()
	df["id"] = df["id"].astype('Int64')

	df.columns = [fnSentenceCase(c) for c in df.columns] 

	display(df.dtypes)
	display(df)

Id                                 Int64
Key                       string[python]
Components Id             string[python]
Components Name           string[python]
Components Description    string[python]
dtype: object

,Id,Key,Components Id,Components Name,Components Description
0,50738,SAT-67,12444,Meeting,<NA>
0,50729,SAT-58,12436,Whitepaper,<NA>
1,50733,SAT-62,12436,Whitepaper,<NA>
1,50785,SAT-106,12440,RADS,Rapid Architecture and Delivery Strategy
1,50785,SAT-106,12442,Report,<NA>
...,...,...,...,...,...
25,46063,HYP-15,12164,Sales,<NA>
25,46064,HYP-13,12161,Marketing,<NA>
25,46064,HYP-13,12164,Sales,<NA>
26,46086,HYP-11,12165,User Experience,<NA>


In [38]:
# Issues Task Linked Issues

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC" 

fields = fnGetIssueTypeFields(ExpectedIssueTypes)
fields = fields[ fields['name'] == 'Linked Issues']

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
else:
	df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
	df = df.explode("issues")
	df = fnExpandColumn(df, "issues", None)
	df = df.drop(["expand", "self"], axis=1)
	df = fnExpandColumn(df, "fields", None)
	df = df[df['issuelinks'].apply(lambda x: len(x)) > 0]
	df = df.explode("issuelinks")
	df = fnExpandColumn(df, "issuelinks")
	df = df.drop(["issuelinks self"], axis=1)
	df = df.convert_dtypes().infer_objects()
	df["id"] = df["id"].astype('Int64')

	df.columns = [fnSentenceCase(c) for c in df.columns] 

	display(df.dtypes)
	display(df)

Id                                   Int64
Key                         string[python]
Issuelinks Id               string[python]
Issuelinks Type                     object
Issuelinks Inward Issue             object
Issuelinks Outward Issue            object
dtype: object

,Id,Key,Issuelinks Id,Issuelinks Type,Issuelinks Inward Issue,Issuelinks Outward Issue
0,50729,SAT-58,36055,"{'id': '10000', 'name': 'Blocks', 'inward': 'i...","{'id': '50127', 'key': 'MKTG-638', 'self': 'ht...",NaN
4,50732,SAT-61,36031,"{'id': '10407', 'name': 'Parent-Child', 'inwar...",NaN,"{'id': '50749', 'key': 'SAT-78', 'self': 'http..."
4,50749,SAT-78,36031,"{'id': '10407', 'name': 'Parent-Child', 'inwar...","{'id': '50732', 'key': 'SAT-61', 'self': 'http...",NaN
46,44063,TRC-18,31497,"{'id': '10400', 'name': 'Extends', 'inward': '...",NaN,"{'id': '44062', 'key': 'TRC-17', 'self': 'http..."


In [7]:
# Issues Task Sprints

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") and sprint is not empty ORDER BY updatedDate DESC" 

fields = fnGetIssueTypeFields(ExpectedIssueTypes)
fields = fields[ fields['name'] == 'Sprint']

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
else:
	df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
	df = df.explode("issues")
	df = fnExpandColumn(df, "issues", None)
	df = df.drop(["expand", "self"], axis=1)
	df = fnExpandColumn(df, "fields", None)
	df = df.rename( columns=dict( zip ( fields.fieldId, fields.name )) )
	df = df.convert_dtypes().infer_objects()
	df["id"] = df["id"].astype('Int64')

	display(df.dtypes)
	display(df)

'No results'

In [45]:
# Issues Task Attachments

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") and attachments is not empty ORDER BY updatedDate DESC" 

fields = fnGetIssueTypeFields(ExpectedIssueTypes)
fields = fields[ fields['name'] == 'Attachment']

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
else:
	df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
	df = df.explode("issues")
	df = fnExpandColumn(df, "issues", None)
	df = df.drop(["expand", "self"], axis=1)
	df = fnExpandColumn(df, "fields", None)
	df = df.explode("attachment")
	df = fnExpandColumn(df, "attachment")
	df = df.convert_dtypes().infer_objects()
	df["id"] = df["id"].astype('Int64')

	display(df.dtypes)
	display(df)

id                               Int64
key                     string[python]
attachment self         string[python]
attachment id           string[python]
attachment filename     string[python]
attachment author               object
attachment created      string[python]
attachment size                  Int64
attachment mimeType     string[python]
attachment content      string[python]
attachment thumbnail    string[python]
dtype: object

,id,key,attachment self,attachment id,attachment filename,attachment author,attachment created,attachment size,attachment mimeType,attachment content,attachment thumbnail
0,50733,SAT-62,https://strategenics.atlassian.net/rest/api/2/...,38122,300523_Enterprise Operating Models v0.3_PD.docx,{'self': 'https://strategenics.atlassian.net/r...,2023-07-21T09:33:23.992+1000,118946,application/vnd.openxmlformats-officedocument....,https://strategenics.atlassian.net/rest/api/2/...,<NA>
0,43934,TASS-5,https://strategenics.atlassian.net/rest/api/2/...,33369,image-20210113-060841.png,{'self': 'https://strategenics.atlassian.net/r...,2021-01-13T16:12:50.801+1000,56712,image/png,https://strategenics.atlassian.net/rest/api/2/...,https://strategenics.atlassian.net/rest/api/2/...
1,44924,TASS-6,https://strategenics.atlassian.net/rest/api/2/...,34291,image-2021-05-28-12-08-59-051.png,{'self': 'https://strategenics.atlassian.net/r...,2021-05-28T12:09:00.698+1000,275466,image/png,https://strategenics.atlassian.net/rest/api/2/...,https://strategenics.atlassian.net/rest/api/2/...
1,43454,TASS-2,https://strategenics.atlassian.net/rest/api/2/...,32804,TASS Help Center v0.1.png,{'self': 'https://strategenics.atlassian.net/r...,2020-05-12T13:39:33.826+1000,384948,image/png,https://strategenics.atlassian.net/rest/api/2/...,https://strategenics.atlassian.net/rest/api/2/...
2,43453,TASS-1,https://strategenics.atlassian.net/rest/api/2/...,32806,image-20200519-013611.png,{'self': 'https://strategenics.atlassian.net/r...,2020-05-19T11:36:24.093+1000,68784,image/png,https://strategenics.atlassian.net/rest/api/2/...,https://strategenics.atlassian.net/rest/api/2/...
2,45540,TRC-346,https://strategenics.atlassian.net/rest/api/2/...,35158,DOCS-#7211062-v1-Minor_Tier_Project_Monthly_Re...,{'self': 'https://strategenics.atlassian.net/r...,2021-12-14T16:24:47.864+1000,242120,application/pdf,https://strategenics.atlassian.net/rest/api/2/...,<NA>
3,45542,TRC-348,https://strategenics.atlassian.net/rest/api/2/...,35157,image-20211214-054555.png,{'self': 'https://strategenics.atlassian.net/r...,2021-12-14T15:46:13.310+1000,834808,image/png,https://strategenics.atlassian.net/rest/api/2/...,https://strategenics.atlassian.net/rest/api/2/...
3,45541,TRC-347,https://strategenics.atlassian.net/rest/api/2/...,35156,image-20211214-054347.png,{'self': 'https://strategenics.atlassian.net/r...,2021-12-14T15:44:12.667+1000,30468,image/png,https://strategenics.atlassian.net/rest/api/2/...,https://strategenics.atlassian.net/rest/api/2/...
4,45445,TRC-344,https://strategenics.atlassian.net/rest/api/2/...,34941,image-20211110-024420.png,{'self': 'https://strategenics.atlassian.net/r...,2021-11-10T13:01:53.171+1000,158202,image/png,https://strategenics.atlassian.net/rest/api/2/...,https://strategenics.atlassian.net/rest/api/2/...
4,45445,TRC-344,https://strategenics.atlassian.net/rest/api/2/...,34944,image-20211110-221645.png,{'self': 'https://strategenics.atlassian.net/r...,2021-11-11T08:17:28.531+1000,30558,image/png,https://strategenics.atlassian.net/rest/api/2/...,https://strategenics.atlassian.net/rest/api/2/...
